In [ ]:
!pip install nltk

In [ ]:
#packages and modules
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
#data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
data=pd.read_csv('/content/Reviews.csv', on_bad_lines='skip',engine='python')
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
data['Text']

,Text
0,I have bought several of the Vitality canned d...
1,Product arrived labeled as Jumbo Salted Peanut...
2,This is a confection that has been around a fe...
3,If you are looking for the secret ingredient i...
4,Great taffy at a great price. There was a wid...
...,...
88955,"I purchased the ""Organic"" variety of this bran..."
88956,"Because we were having guests for a visit, I o..."
88957,Coffee People's Tree Hugger Coffee is the favo...
88958,"Great coffee, and I feel good knowing it's fai..."


In [ ]:
data.isna().sum()

,0
Id,0
ProductId,0
UserId,0
ProfileName,6
HelpfulnessNumerator,0
HelpfulnessDenominator,0
Score,0
Time,0
Summary,2
Text,0


In [ ]:
data.dropna(inplace=True)

In [ ]:
processed_data=data[:10000]

In [ ]:
stop_words=set(stopwords.words('english'))
print(stop_words)

{'further', 're', "she'd", 'other', 'nor', "she'll", 'our', 'where', "you'd", "aren't", 'does', 'during', 'll', 'then', 'your', 'do', "he'd", "we'll", 'this', 'any', 'couldn', 'y', "couldn't", 'same', 'a', 'them', "weren't", 'above', 'wouldn', 'not', 'both', 'there', "they'd", 'o', 'no', 'again', "it'll", "they'll", 'as', 'most', 'are', "should've", "we've", 'shouldn', 'from', 'been', 'i', "they're", 'each', 'ma', 'd', "doesn't", 'shan', 'yourself', 'ain', "i'd", 'be', 'after', 'haven', 'below', "hasn't", "wouldn't", "shan't", 'yours', 'mustn', 'she', 'before', 'he', 'needn', 'were', 'her', 'those', 'why', "wasn't", "that'll", 'under', "mightn't", 'my', 'than', 'with', "won't", "they've", 'too', "it'd", 'but', 'or', 'all', 'of', "shouldn't", 'yourselves', 'weren', "isn't", 'the', 'himself', 'who', 'how', "he's", 'through', 'did', "needn't", 'that', 'hasn', 've', 'its', 'themselves', 'him', 'mightn', 'own', "you'll", 'in', 'itself', 'doing', 'an', "i've", 'more', 'is', "don't", 'having'

In [ ]:
def cleaned_text(text):
  text=text.lower()
  text = re.sub(r'[^a-z\s]', '', text)
  tokens=word_tokenize(text)
  tokens=[word for word in tokens if word not in stop_words]
  cleaned_text=' '.join(tokens)
  return cleaned_text

In [ ]:
import re
data['cleaned_text']=data['Text'].apply(cleaned_text)

In [ ]:
print(data['cleaned_text'].head())

0    bought several vitality canned dog food produc...
1    product arrived labeled jumbo salted peanutsth...
2    confection around centuries light pillowy citr...
3    looking secret ingredient robitussin believe f...
4    great taffy great price wide assortment yummy ...
Name: cleaned_text, dtype: object


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
texts=data['cleaned_text']
tfidf=TfidfVectorizer(max_features=5000)
tfidf_fit=tfidf.fit_transform(texts)
tfidf_fit.shape

(88952, 5000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def retrive_reviews(query,k=5):
  cleaned_query=query.lower()
  cleaned_query=cleaned_text(query)
  query_vec=tfidf.transform([cleaned_query])
  similarity_score=cosine_similarity(query_vec,tfidf_fit).flatten()
  top_indices=similarity_score.argsort()[::-1][:k]

  results=[]
  for i in top_indices:
    results.append({
        "original":data.iloc[i]['Text'],
        "cleaned":data.iloc[i]['cleaned_text'],
        "score":float(similarity_score[i])
    })
  return results

In [ ]:
for res in retrive_reviews("great product with fast fishing", k=3):
    print("Original:", res["original"])
    print("Cleaned :", res["cleaned"])
    print("Score   :", res["score"])
    print("-" * 80)


Original: Great gift. Fast delivery. Quality product.
Cleaned : great gift fast delivery quality product
Score   : 0.5815210599574625
--------------------------------------------------------------------------------
Original: I have enjoyed this product very much and the price is great and the delivery is fast.
Cleaned : enjoyed product much price great delivery fast
Score   : 0.5765251384431493
--------------------------------------------------------------------------------
Original: Love the taste of this coffee. Great flavor- It's not too strong like some other products for Keurig!<br />i certainly would recommend this product. Shipping was great. Fast!
Cleaned : love taste coffee great flavor strong like products keurigbr certainly would recommend product shipping great fast
Score   : 0.5093048893417348
--------------------------------------------------------------------------------


In [ ]:
for res in retrive_reviews("disappointed", k=3):
    print("Original:", res["original"])
    print("Cleaned :", res["cleaned"])
    print("Score   :", res["score"])
    print("-" * 80)


Original: Was disappointed with product.  The sugar is all clumped up - will not order this product again.
Cleaned : disappointed product sugar clumped order product
Score   : 0.5249478534184538
--------------------------------------------------------------------------------
Original: I am a bit disappointed.  The flavor was not what I wanted or expected.
Cleaned : bit disappointed flavor wanted expected
Score   : 0.48530831783512385
--------------------------------------------------------------------------------
Original: Extremely disappointed in the quality of this brand's brewing quality.  Will not order this particular brand again.  Will dispose of, instead of being disappointed the majority of the time.
Cleaned : extremely disappointed quality brands brewing quality order particular brand dispose instead disappointed majority time
Score   : 0.4738175263390573
--------------------------------------------------------------------------------
